## Keras Tuner

In [1]:
!pip install keras-tuner
!pip install scipy
import scipy

  Using cached keras_tuner-1.3.5-py3-none-any.whl (176 kB)
  Using cached kt_legacy-1.0.5-py3-none-any.whl (9.6 kB)
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/a3/d3/f88285098505c8e5d141678a24bb9620d902c683f11edc1eb9532b02624e/scipy-1.11.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached scipy-1.11.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (59 kB)
Using cached scipy-1.11.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.5 MB)


## Parameters

In the course instructor used this parameters:

learning rate: [0.0001, 0.001, 0.01, 0.1]

sizes of inner layer [10, 100, 1000],

droprate = [0.0, 0.2, 0.5, 0.8]



In [2]:
import keras_tuner
from keras_tuner.tuners import RandomSearch, Hyperband

2023-09-03 14:04:54.264785: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
import tensorflow as tf

### Importing Pre-trained convolutional neural networks

In [4]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow import keras

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Spliting Dataset

In [6]:


train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(299, 299),
    batch_size=32
)


val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(299, 299),
    batch_size=32,
    shuffle=False
)



Found 3068 images belonging to 10 classes.
Found 341 images belonging to 10 classes.


## Model Function

In [7]:
from tensorflow.keras import backend as c

In [8]:
def make_model(hp):
    c.clear_session()
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(299, 299, 3)
    )
    base_model.trainable = False
    #########################################
    inputs = keras.Input(shape=(299, 299, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    # Size Inner Block
    size_inner= hp.Choice("size_inner", values = [10, 100, 1000])
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    
    #Droprate Block
    droprate = hp.Choice("droprate", values= [0.0, 0.2, 0.5, 0.8]) # As an alternaive droprate = hp.Int("droprate", min_value=0.0, max_value=0.8, step=0.2)
    drop = keras.layers.Dropout(droprate)(inner)


    
    outputs = keras.layers.Dense(10)(drop)
    
    model = keras.Model(inputs, outputs)
    
    #########################################
    # Learning Rate Block
    learning_rate = hp.Choice("learning_rate", values=[0.0001, 0.001, 0.01, 0.1])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )

    
    return model

### Tensorboard Callback

In [9]:
log_dir="/home/jovyan/workspace/logs/fit"
callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=0,
write_graph=True,
write_images=False,
write_steps_per_second=False,
update_freq='epoch',
profile_batch=0,
embeddings_freq=0,
embeddings_metadata=None,)

### Tuner RandomSearch

In [10]:
tuner = RandomSearch(
    make_model,
    objective='val_accuracy',
    max_trials=25,
    executions_per_trial=1,
    directory='my_dir1') #change the directory name here  when rerunning the cell else it gives "Oracle exit error" 

tuner.search_space_summary()

2023-09-03 14:05:00.850037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-03 14:05:00.856826: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-03 14:05:00.857089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-03 14:05:00.857661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Search space summary
Default search space size: 3
size_inner (Choice)
{'default': 10, 'conditions': [], 'values': [10, 100, 1000], 'ordered': True}
droprate (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.2, 0.5, 0.8], 'ordered': True}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.001, 0.01, 0.1], 'ordered': True}


In [11]:
tuner.search(train_ds, epochs=20, validation_data=val_ds,
                   callbacks=[callback])

Trial 20 Complete [00h 22m 56s]
val_accuracy: 0.8856304883956909

Best val_accuracy So Far: 0.900293231010437
Total elapsed time: 07h 35m 27s
INFO:tensorflow:Oracle triggered exit


In [12]:
tuner.results_summary()

Results summary
Results in my_dir1/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 14 summary
Hyperparameters:
size_inner: 100
droprate: 0.5
learning_rate: 0.001
Score: 0.900293231010437

Trial 09 summary
Hyperparameters:
size_inner: 1000
droprate: 0.8
learning_rate: 0.001
Score: 0.8914955854415894

Trial 07 summary
Hyperparameters:
size_inner: 100
droprate: 0.0
learning_rate: 0.001
Score: 0.8885630369186401

Trial 15 summary
Hyperparameters:
size_inner: 100
droprate: 0.2
learning_rate: 0.01
Score: 0.8856304883956909

Trial 17 summary
Hyperparameters:
size_inner: 100
droprate: 0.8
learning_rate: 0.001
Score: 0.8856304883956909

Trial 19 summary
Hyperparameters:
size_inner: 1000
droprate: 0.0
learning_rate: 0.001
Score: 0.8856304883956909

Trial 10 summary
Hyperparameters:
size_inner: 1000
droprate: 0.2
learning_rate: 0.01
Score: 0.873900294303894

Trial 13 summary
Hyperparameters:
size_inner: 1000
droprate: 0.2
learning_rate: 0.001
Score: 

### Listing Best Hyperparameters

In [17]:
best_params = tuner.get_best_hyperparameters(1)[0]
print(best_params.values)

{'size_inner': 100, 'droprate': 0.5, 'learning_rate': 0.001}


### Early Stop

In [24]:
early_stop =tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=17,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=False,
)


In [26]:
best_model = tuner.hypermodel.build(best_params)



In [28]:
history = best_model.fit(train_ds, epochs=50, validation_data=val_ds,
                   callbacks=[callback, early_stop])

Epoch 1/50
96/96 [==============================] - 69s 714ms/step - loss: 0.5385 - accuracy: 0.8256 - val_loss: 0.4309 - val_accuracy: 0.8739
Epoch 2/50
96/96 [==============================] - 69s 720ms/step - loss: 0.4788 - accuracy: 0.8357 - val_loss: 0.4289 - val_accuracy: 0.8739
Epoch 3/50
96/96 [==============================] - 69s 721ms/step - loss: 0.4271 - accuracy: 0.8559 - val_loss: 0.3897 - val_accuracy: 0.8680
Epoch 4/50
96/96 [==============================] - 68s 706ms/step - loss: 0.3934 - accuracy: 0.8677 - val_loss: 0.3607 - val_accuracy: 0.8739
Epoch 5/50
96/96 [==============================] - 69s 720ms/step - loss: 0.3801 - accuracy: 0.8693 - val_loss: 0.3821 - val_accuracy: 0.8563
Epoch 6/50
96/96 [==============================] - 68s 706ms/step - loss: 0.3445 - accuracy: 0.8833 - val_loss: 0.3553 - val_accuracy: 0.8827
Epoch 7/50
96/96 [==============================] - 68s 707ms/step - loss: 0.3252 - accuracy: 0.8862 - val_loss: 0.3716 - val_accuracy: 0.8739